In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_22016\3609341226.py:20: DeprecationWarning:

`set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`



{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1647454514.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
from pandas_datareader import DataReader as pdr
import plotly.express as px
rets = pdr(
  ['cvx', 'tsla'], 
  "yahoo", 
  start="2016-12-01", 
  end="2021-12-31"
)['Adj Close']  
rets = rets.resample('M').last().pct_change().dropna() 
rets.index = rets.index.to_period('M').astype(str)
rets = rets.reset_index()
rets.columns = ['date', 'CVX', 'TSLA'] 

fig = px.scatter(
        rets,
        x="CVX",
        y="TSLA",
        trendline="ols",
        hover_data=dict(TSLA=False, CVX=False, date=False),
        hover_name="date",
    )

fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title="CVX Return",
    yaxis_title="TSLA Return",
    yaxis_tickformat=".0%",
    xaxis_tickformat=".0%",
    width=520,
    height=520,
  )
fig.update_xaxes(title_font_size=24)
fig.update_yaxes(title_font_size=24)
fig.update_layout(font_size=20)
fig.show()

In [3]:
print(f"std dev of CVX is {rets.CVX.std(): .2%}")
print(f"std dev of TSLA is {rets.TSLA.std(): .2%}")
print(f"corr is {rets.TSLA.corr(rets.CVX): .3f}")
print(f"cov is {100*100*rets.TSLA.cov(rets.CVX): .1f}%^2")
print(f"beta is {rets.TSLA.cov(rets.CVX)/rets.CVX.var(): .2f}")

std dev of CVX is  8.18%
std dev of TSLA is  19.61%
corr is  0.243
cov is  39.0%^2
beta is  0.58


In [4]:
from pandas_datareader import DataReader as pdr
import plotly.express as px
rets = pdr(
  ['cvx', 'xom'], 
  "yahoo", 
  start="2016-12-01", 
  end="2021-12-31"
)['Adj Close']  
rets = rets.resample('M').last().pct_change().dropna() 
rets.index = rets.index.to_period('M').astype(str)
rets = rets.reset_index()
rets.columns = ['date', 'CVX', 'XOM'] 

fig = px.scatter(
        rets,
        x="CVX",
        y="XOM",
        trendline="ols",
        hover_data=dict(XOM=False, CVX=False, date=False),
        hover_name="date",
    )

fig.update_traces(
        marker=dict(size=12, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
)

fig.update_layout(
    template="plotly_dark",
    xaxis_title="CVX Return",
    yaxis_title="XOM Return",
    yaxis_tickformat=".0%",
    xaxis_tickformat=".0%",
    width=520,
    height=520,
  )
fig.update_xaxes(title_font_size=24)
fig.update_yaxes(title_font_size=24)
fig.update_layout(font_size=20)
fig.show()

In [5]:
print(f"std dev of CVX is {rets.CVX.std(): .2%}")
print(f"std dev of XOM is {rets.XOM.std(): .2%}")
print(f"corr is {rets.XOM.corr(rets.CVX): .3f}")
print(f"cov is {100*100*rets.XOM.cov(rets.CVX): .1f}%^2")
print(f"beta is {rets.XOM.cov(rets.CVX)/rets.CVX.var(): .2f}")

std dev of CVX is  8.18%
std dev of XOM is  8.58%
corr is  0.889
cov is  62.4%^2
beta is  0.93
